# Server

In [3]:
import tekore as tk
from dotenv import load_dotenv

from authentication.spotify_server import SpotifyServer

load_dotenv()

conf = tk.config_from_environment()
cred = tk.Credentials(*conf)
spotify = tk.Spotify()
host = '127.0.0.1'
port = 5000

app = SpotifyServer(host, port, spotify, cred)
spotify.token = app.spawn_single_use_server()

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Jan/2023 14:27:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2023 14:27:59] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Jan/2023 14:28:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2023 14:28:01] "GET /login HTTP/1.1" 307 -


# Spotify API

In [4]:
spotify.token = cred.refresh(spotify.token)

In [5]:
from project.random_track import RandomTrack

rndTrack = RandomTrack(spotify, cred)
tracks = rndTrack.random_tracks()
for track in tracks:
    print(track.name)

The King Welcomes You to Avatar Country
KILL ALL PREDATORS
Release The Kraken - Sefa Remix
Kingdom Come
A Statue of the King
KICK BACK
Kryptonite
Hail and Kill
Killing In The Name
K.
Das ist alles von der Kunstfreiheit gedeckt
The Kids Aren't Alright
Kids
You're Gonna Go Far, Kid
Kalasi
Kokain
Murder In My Mind
One Kiss (with Dua Lipa)
Unholy (feat. Kim Petras)
Kernkraft 400 (A Better Day)
Danza Kuduro
Karamela
kaleidoskop
Kings & Queens
King Kong
Karneval
Memories (feat. Kid Cudi)
Timber (feat. Ke$ha)
Danny Don't You Know
Kill Bill
Kein Problem
Somewhere Only We Know
Kontrollieren
Follow
Kiss Me
KU LO SA - A COLORS SHOW
lovely (with Khalid)
Krankenwagen
Kleiner Prinz
Keine Tränen
KISSENSCHLACHT
Kleiner Finger Schwur
Kiss Me
Kid On Christmas (feat. Meghan Trainor)
Do They Know It's Christmas? - 1984 Version
Do They Know It's Christmas? - 2014
Underneath the Tree
Ticket nach Ketama (feat. Liaze)
Often - Kygo Remix
Alright (feat. KIDDO)


In [6]:
from project.util import Partition

artist_id_limit=50
artist_ids = [artist.id for track in tracks for artist in track.artists]
artist_id_partitions = Partition(artist_ids)
artists = [spotify.artists(ids) for ids in artist_id_partitions]
artists = [artist for partition in artists for artist in partition]
artists

[FullArtist with fields:
   external_urls = {'spotify'}
   followers = Followers(href, total)
   genres = [5 x str]
   href = 'https://api.spotify.com/v1/artists/4jpaXieuls7LVzG1uma5Rs'
   id = '4jpaXieuls7LVzG1uma5Rs'
   images = [3 x Image(height, url, width)]
   name = 'Avatar'
   popularity = 57
   type = 'artist'
   uri = 'spotify:artist:4jpaXieuls7LVzG1uma5Rs',
 FullArtist with fields:
   external_urls = {'spotify'}
   followers = Followers(href, total)
   genres = [1 x str]
   href = 'https://api.spotify.com/v1/artists/0DG7J8Q9Alnt65HJv6owzf'
   id = '0DG7J8Q9Alnt65HJv6owzf'
   images = [3 x Image(height, url, width)]
   name = 'Banshee'
   popularity = 51
   type = 'artist'
   uri = 'spotify:artist:0DG7J8Q9Alnt65HJv6owzf',
 FullArtist with fields:
   external_urls = {'spotify'}
   followers = Followers(href, total)
   genres = [4 x str]
   href = 'https://api.spotify.com/v1/artists/0zCrrgklotTrkkJDNUAury'
   id = '0zCrrgklotTrkkJDNUAury'
   images = [3 x Image(height, url, widt

In [7]:
import pylast
from authentication.lastfm_credentials import LastFmCredentials
from lastfm import LastFmScraper, LastFmProxy

creds = LastFmCredentials("f06f4002f217a1c17ca7864a15fe059f", "00f207939f38850e1870131a03bf406a")
last_network = pylast.LastFMNetwork(
    api_key=creds.api_key,
    api_secret=creds.shared_secret,
)

lastfm = LastFmProxy(last_network, LastFmScraper())

In [14]:
import track as model
analyzed_tracks = model.AnalyzedTracks(tracks, artists)

[FullArtist with fields:
  external_urls = {'spotify'}
  followers = Followers(href, total)
  genres = [5 x str]
  href = 'https://api.spotify.com/v1/artists/4jpaXieuls7LVzG1uma5Rs'
  id = '4jpaXieuls7LVzG1uma5Rs'
  images = [3 x Image(height, url, width)]
  name = 'Avatar'
  popularity = 57
  type = 'artist'
  uri = 'spotify:artist:4jpaXieuls7LVzG1uma5Rs', FullArtist with fields:
  external_urls = {'spotify'}
  followers = Followers(href, total)
  genres = [1 x str]
  href = 'https://api.spotify.com/v1/artists/0DG7J8Q9Alnt65HJv6owzf'
  id = '0DG7J8Q9Alnt65HJv6owzf'
  images = [3 x Image(height, url, width)]
  name = 'Banshee'
  popularity = 51
  type = 'artist'
  uri = 'spotify:artist:0DG7J8Q9Alnt65HJv6owzf', FullArtist with fields:
  external_urls = {'spotify'}
  followers = Followers(href, total)
  genres = [4 x str]
  href = 'https://api.spotify.com/v1/artists/0zCrrgklotTrkkJDNUAury'
  id = '0zCrrgklotTrkkJDNUAury'
  images = [3 x Image(height, url, width)]
  name = 'ZAND'
  popula

In [9]:
from project.track import TaggedTrack
# TODO: fix empty artists
for track in analyzed_tracks.tracks:
    try:
        tags = TaggedTrack(lastfm, track.name, track.artist_names).tags()
        if len(tags) == 0:
            raise Exception("No tags")
    except Exception as e:
        print(e, track.id, track.name, track.artist_names)
        continue
    track.tags = tags

No tags 7j1ZJAbcm4AMBf82jIrAH3 Kiss Me []
No tags 2WigMwGJysIh9fRnSJvpjn KU LO SA - A COLORS SHOW []
No tags 0u2P5u6lvoDfwTYjAADbn4 lovely (with Khalid) []
No tags 6evJ8sysntgtBBBs6WSxqQ Krankenwagen []
No tags 0y8lwLPXBsenbF5WAZhlbL Kleiner Prinz []
No tags 03ClGz3IWepPxLcVlGSC5Q Keine Tränen []
No tags 5s1EwpwmgQJ8Ti3ltrJdO8 KISSENSCHLACHT []
No tags 3PBPLps3EbPjBSSOpKmJ8J Kleiner Finger Schwur []
No tags 76SBSmc38qQQSpkgBenb8U Kiss Me []
No tags 5glU2EWqa5hpYqGPboSNjV Kid On Christmas (feat. Meghan Trainor) []
No tags 4Yg2w0P29BBBMixyeNjDtj Do They Know It's Christmas? - 1984 Version []
No tags 3iMRGDhn0ghHIUMmlKrJOh Do They Know It's Christmas? - 2014 []
No tags 3nAp4IvdMPPWEH9uuXFFV5 Underneath the Tree []
No tags 7KWwQ8SokgUDO7es6Q4ZGW Ticket nach Ketama (feat. Liaze) []
No tags 3TZgqQklJrSBX31Y0UIIUL Often - Kygo Remix []
No tags 1vUOJpWc091SBYULNvz9mp Alright (feat. KIDDO) []


# Firebase

In [12]:
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.ApplicationDefault()
firebase_admin.initialize_app(cred)

In [13]:
from google.cloud.firestore_v1 import Client

firestore_client: Client = firestore.client()
analyzed_tracks.upsert(firestore_client.collection("tracks"))